# Synchronization of Threads Using Barrier and Ordered Directive

## Introduction

OpenMP provides a selection of tools to achieve synchronization between threads, ensuring smooth collaboration and preventing the pitfalls of concurrent execution. Among these, the `barrier` and `ordered` directives stand as pillars of thread coordination, each playing a distinct role in maintaining order and predictability within parallel regions.

- **Barrier Directive**: Acting as a synchronization point, the barrier directive ensures all threads within a team reach a designated point before any are allowed to proceed. This is crucial in preventing race conditions, where multiple threads accessing and modifying shared data simultaneously can lead to unpredictable and erroneous results.

- **Ordered Directive**: For scenarios where the order of execution is critical, the ordered directive ensures specific sections within a parallelized loop are executed sequentially, respecting the order of iterations. This is essential for maintaining deterministic behavior in situations like I/O operations or when data dependencies exist between iterations.

## Barrier Directive

The `barrier` directive acts as a crucial tool for orchestrating thread synchronization within a parallel region. It establishes designated points where all threads within a team must converge before any are permitted to proceed. This mechanism is vital for preventing race conditions and maintaining data consistency in shared-memory parallel programs, ensuring predictable and reliable results.

### Establishing Synchronization Points

The syntax for employing the barrier directive is straightforward and consistent across both C/C++ and Fortran:

**C/C++:**

```
#pragma omp barrier
```

**Fortran:**

```
!$omp barrier
```

By strategically placing this directive within a parallel region, you create explicit synchronization points. Let's delve into a concrete example to illustrate this:

```
#pragma omp parallel
{
  // Each thread performs its independent calculations on a portion of data...
  #pragma omp barrier
  // All threads synchronize here, ensuring consistent data before the next step
  // Combine the partial results from each thread into a final result...
}
```

In this scenario, each thread works on its assigned portion of data before encountering the barrier. The `barrier` directive acts as a gatekeeper, halting each thread until all threads within the team have reached this point. This guarantees that all partial calculations are complete and the shared data is in a consistent state before moving on to combine the partial results.

### Example

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

#include <omp.h>
#include <stdio.h>

int main() {
    int sum = 0;
    #pragma omp parallel shared(sum)
    {
        int tid = omp_get_thread_num();
        int local_sum = 0;
        // Each thread calculates a local sum for a portion of data
        for (int i = tid * 10; i < (tid + 1) * 10; ++i) {
            local_sum += i;
        }
        
        #pragma omp critical
        sum += local_sum; // Add each thread's local sum to the global sum
        
        #pragma omp barrier // All threads wait here before printing
        
        printf("Thread %d finished with local sum %d\n", tid, local_sum);
    }
    printf("Final sum: %d\n", sum);
    return 0;
}

## The Ordered Directive: Maintaining Sequential Steps

In the realm of parallelized loops, where iterations dance across threads, maintaining a specific execution order for certain sections of code becomes essential. The ordered directive steps onto the stage, ensuring that designated portions of a loop are executed sequentially, respecting the order of iterations even when the loop itself is parallelized and iterations might be executed concurrently.

### Enforcing Order in the Parallel Ballet

The syntax for invoking the ordered directive is clear and consistent across C/C++ and Fortran:

**C/C++:**

```
#pragma omp ordered
structured-block
#pragma omp end ordered
```

**Fortran:**
```
!$omp ordered
structured-block
!$omp end ordered
```

Let's explore a simple example to grasp the essence of the ordered directive:

```
#pragma omp parallel for ordered
for (int i = 0; i < N; ++i) {
  // Perform calculations specific to iteration i...
  #pragma omp ordered
  printf("Iteration %d from thread %d\n", i, omp_get_thread_num());
}
```

In this scenario, even though the loop iterations might be scattered across threads and executed concurrently, the printf statement within the ordered region will always print in the order of the loop iterations (0, 1, 2, ...). The ordered directive guarantees that each thread executes this section sequentially, respecting the natural order of the loop.
### doacross Clause: Specifying Dependencies
For more intricate scenarios involving loops with cross-iteration dependencies, the doacross clause comes to the rescue. It allows you to pinpoint specific dependences between iterations that must be honored during parallel execution. By using the doacross clause, you provide explicit instructions to the compiler regarding the required order of operations, ensuring correctness even in the presence of complex data dependencies.

## Implicit Barriers: Automatic Synchronization

OpenMP provides automatic synchronization with implicit barriers placed at the end of worksharing constructs (like for and sections) and at the end of parallel regions. These invisible barriers guarantee that all threads have finished their assigned work within the construct before moving on to the next section of code.
**Example:**
```
#pragma omp parallel for
for (int i = 0; i < N; ++i) {
    // Work on iteration i...
}
// Implicit barrier - all threads wait here before proceeding
```

### Removing Implicit Barriers with nowait

While implicit barriers provide convenient synchronization, they might introduce unnecessary overhead in certain scenarios. The nowait clause allows you to remove these implicit barriers, enabling threads to proceed without waiting for others. However, use this clause judiciously and ensure proper explicit synchronization mechanisms are in place to avoid race conditions.

**Example:**

```
#pragma omp parallel for nowait
for (int i = 0; i < N; ++i) {
    // Work on iteration i...
}
// No implicit barrier - threads continue immediately
```


## Best Practices for Using Barrier and Ordered Directives: Achieving Synchronization Zen
While the barrier and ordered directives are powerful tools for orchestrating thread synchronization, their effectiveness hinges on understanding their nuances and applying them strategically. Here are some guiding principles to help you achieve synchronization zen and unlock the full potential of these directives:
- Barrier Mindfulness: Barriers, while essential for preventing race conditions, can introduce overhead due to thread idling and synchronization costs. Use them judiciously and only when necessary to ensure data consistency and correctness. Explore alternative synchronization mechanisms like locks or atomic operations when appropriate.
- Embrace Implicit Barriers: Leverage the convenience and efficiency of implicit barriers automatically inserted at the end of worksharing constructs and parallel regions. Be mindful of their presence and consider the nowait clause only when you are confident about the absence of race conditions and its impact on correctness.
- Order with Purpose: Apply the ordered directive selectively to code sections within a loop where maintaining the order of execution is critical. Overusing the ordered directive can diminish the benefits of parallelization, so carefully analyze your code's data dependencies and apply it only when necessary.
- Data Dependencies Awareness: Gain a deep understanding of the data dependencies within your code, both within iterations and across iterations. This knowledge is crucial for selecting the appropriate synchronization directives and ensuring that threads access and modify shared data in a controlled manner.
- Performance as the Guiding Light: Employ performance profiling tools to identify and address potential bottlenecks arising from synchronization overhead. Optimize your code by minimizing unnecessary barriers and restructuring your loops to reduce cross-iteration dependencies, striking a balance between synchronization and parallel efficiency.